In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import current_timestamp

source_df = spark.table("bronze_db.orders")

if not spark.catalog.tableExists("silver_db.orders"):
    silver_df = (
        source_df
        .withColumn("created_ts", current_timestamp())
        .withColumn("modified_ts", current_timestamp())
    )

    silver_df.write.format("delta").saveAsTable("silver_db.orders")

else:
    target = DeltaTable.forName(spark, "silver_db.orders")

    staged_df = source_df.withColumn("etl_ts", current_timestamp())

    (
        target.alias("t")
        .merge(
            staged_df.alias("s"),
            "t.order_id = s.order_id"
        )
        .whenMatchedUpdate(
            condition="""
        NOT (
            t.status <=> s.status AND
            t.amount <=> s.amount AND
        )
    """,
            set={
            "status": "s.status",
            "amount": "s.amount",
            "modified_ts": "s.etl_ts"
        })
        .whenNotMatchedInsert(values={
            "order_id": "s.order_id",
            "customer_id": "s.customer_id",
            "order_date": "s.order_date",
            "amount": "s.amount",
            "status": "s.status",
            "created_ts": "s.etl_ts",
            "modified_ts": "s.etl_ts"
        })
        .execute()
    )
